# Torre de Hanoi
**Facundo A. Lucianna - Inteligencia Artificial - CEIA - FIUBA**

El rompecabezas comienza con los discos apilados en una varilla en orden de tamaño decreciente, el más pequeño en la parte superior, aproximándose así a una forma cónica. 

El objetivo del rompecabezas es mover toda la pila a una de las otras barras, con las reglas de la leyenda:
1. Sólo se puede mover un disco a la vez.
2. Cada movimiento consiste en coger el disco superior de una de las pilas y colocarlo encima de otra pila o sobre una varilla vacía.
3. Ningún disco podrá colocarse encima de un disco que sea más pequeño que él.

## Resolviendo este problema usando IA

Este problema es un típico problema para aplicar métodos de búsquedas. Podemos crear un agente que pueda resolver este problema. 

El agente puede percibir cuantos discos y en qué orden hay en cada varilla. Además, puede tomar cualquier disco que se encuentre en la parte superior y moverlo a cualquier otra varilla que esté permitido moverlo. 

Definamos el problema para que podamos resolverlo,

### Espacio de estados:

Para 5 discos, tenemos $3^5 = 243$ posibles estados,

![estados_hanoi](./img/state_hanoi1.png)

### Estado inicial

Para este caso arrancamos con todos los discos de mayor a menor en la varilla izquierda.

![estados_hanoi_initial](./img/state_hanoi2.png)

### Estado objetivo

Para simplificar, vamos a tener un solo estado objetivo. Este caso el objetivo es terminar con todos los discos de mayor a menor en la varilla derecha.

![estados_hanoi_goal](./img/state_hanoi3.png)

----

Veamos una implementación de las diferentes estructuras que tenemos definidos del problema de la Torre de Hanoi que vimos en el video de resolución de problemas mediante búsqueda. Para ello nos vamos a basar en el código del libro Artificial Intelligence - A Modern Approach de Norvig And Russell, el cual está [disponible acá](https://github.com/aimacode). Particularmente usamos la [versión de Python](https://github.com/aimacode/aima-python).

OBS: Si deseas armar una versión en otro lenguaje, podes ver desde el link que la librería que nos basamos está escrita en otros lenguajes.
OBS2: Para profundizar no solo en como usarlo, sino en la implementación, se recomienda ver el archivo `hanoi_states.py` en `./lib`, el cual está ampliamente comentado. 

---

# Estado

Podemos representar cualquier estado con la clase `StatesHanoi` 

In [1]:
from aima_libs.hanoi_states import StatesHanoi

Para representar la ubicación de los discos, usamos tres listas, uno por varilla, y un número del 1 al 5 para cada disco. Si queremos representar este estado

![estados_hanoi_goal](./img/state_hanoi0.png)

In [2]:
varilla_izquierda = []
varilla_medio = [5, 3, 1]
varilla_derecha = [4, 2]

In [3]:
state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Con esta clase tenemos la posibilidad de imprimir el estado:

In [4]:
print(state)

HanoiState:  | 5 3 1 | 4 2


Y tenemos los siguientes métodos:

In [5]:
# Podemos ver cual es el disco que está mas arriba de una varilla
disk = state.get_last_disk_rod(number_rod=1)
print(f"El ultimo disco de la varilla del centro es {disk}")

El ultimo disco de la varilla del centro es 1


In [6]:
# Podemos ver si poner un disco en una varilla es un movimiento válido
disk = 1
print("Podemos poner el disco 1 en la varilla derecha?")
if state.check_valid_disk_in_rod(number_rod=2, disk=disk):
    print("Si, es posible poner el disco 1 en la varilla de derecha")

Podemos poner el disco 1 en la varilla derecha?
Si, es posible poner el disco 1 en la varilla de derecha


In [7]:
# Podemos modificar el estado aplicando un movimiento válido
state.put_disk_in_rod(number_rod=2, disk=disk)
print(f"El nuevo estado es: {state}")

El nuevo estado es: HanoiState:  | 5 3 | 4 2 1


El cual no genera el siguiente estado:

![estados_hanoi_goal](./img/state_hanoi4.png)

Esta clase tiene además implementado:

Atributos:
* `rods`: Es la lista de lista con los discos en cada varilla
* `number_of_disks`: El número de discos que hay en esta torre de Hanoi. Estamos usando 5, pero la implementación permite para cualquier número de discos.
* `number_of_pegs`: El número de varillas. Siempre es 3.
* `accumulated_cost`: Es el costo acumulado. Por ahora no tiene sentido este atributo, pero más adelante nos va a servir para ir determinando el costo que nos llevó ir hasta ahí.

Métodos:
* `accumulate_cost`: Es un método que se le pasa un valor de costo y lo va acumulando en el atributo `accumulated_cost`. Similar a `accumulated_cost`, más adelante tendrá más sentido.
* `get_state`: Método getter que nos devuelve el atributo `rods`
* `get_state_dict`: Método getter que nos devuelve el atributo `rods` en forma de diccionario.
* `get_accumulated_cost`: Método getter que nos permite obtener el costo acumulado.
* `check_valid_disk_in_rod`
* `get_last_disk_rod`
* `put_disk_in_rod`

Además tiene implementada métodos que nos permite hacer diferentes operaciones en Python:

* Podemos comparar dos estados (haciendo `estado1 == estado2`)
* Podemos preguntar si un estado es mayor a otro (haciendo `estado1 > estado2`), esto significa si el costo acumulado de un costo es mayor a otro.
* Tenemos una representación en string del estado, y es por eso que cuando hacemos `print()` se observa cómo están los discos.
* También podemos obtener un hash del estado, esto funciona si hacemos `hash(estado)`

---
### Acciones

Además de tener los estados, tenemos las acciones que podemos aplicar para pasar de un estado a otro, es decir mover un disco de una varilla a otra. Para ello tenemos la clase `ActionHanoi`:

In [8]:
from aima_libs.hanoi_states import ActionHanoi

Vamos a aplicar la acción que realiza la acción de ir de este estado:

![estados_hanoi_goal](./img/state_hanoi0.png)

A este estado:

![estados_hanoi_goal](./img/state_hanoi4.png)

Es decir mover el disco 1 a la varilla derecha:

In [9]:
varilla_izquierda = []
varilla_medio = [5, 3, 1]
varilla_derecha = [4, 2]
state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Acá creamos la acción, pero no se aplicó todavia:

In [10]:
action_example = ActionHanoi(disk=1, rod_input=1, rod_out=2)
print(action_example)

Move disk 1 from 2 to 3


Acá aplicamos la acción al estado `state`, el cual nos devuelve un nuevo estado:

In [11]:
new_state = action_example.execute(state_hanoi=state)

print(new_state)

HanoiState:  | 5 3 | 4 2 1


Ahora vemos que el costo acumulado empieza a tener sentido, aplicar la acción de mover un disco tiene un costo igual a 1, entonces el nuevo estado, tiene un costo acumulado de 1:

In [12]:
print(f"El costo acumulado del nuevo estado es {new_state.accumulated_cost}")

El costo acumulado del nuevo estado es 1.0


La clase `ActionHanoi` tiene además implementado:

Atributos:
* `disk`: Es el disco que se quiere mover en la acción
* `rod_input`: Es la varilla de donde sale el disco.
* `rod_out`: Es la varilla de donde sale el disco.
* `action`: Es un string con la acción que se va a aplicar
* `action_dict`: Es un diccionario con toda la información de la acción.
* `cost`: Es el costo de la acción, la cual es siempre igual a 1, salvo cuando el disco se mueve desde y hacia la misma varilla, el cual el costo ahí es 0.

Métodos:
* `execute`: Es el método que aplica la acción, tal como vimos más arriba, toma como entrada el estado que se le quiere aplicar la acción y devuelve un nuevo estado. Además al nuevo estado le suma al costo acumulado el costo de la acción.
  
Además tiene implementada métodos que nos permite hacer diferentes operaciones en Python:

* Tenemos una representación en string del estado, y es por eso que cuando hacemos `print()` se observa una descripción de la acción.

---
### Problema de Hanoi

Por último podemos implementar el problema que tenga todo el problema incorporado, desde un estado inicial, a un estado final, y la posibilidad de movimientos de un estado a otro. De tal forma que podemos movernos por el grafo de estado. Nuesta implementación es `ProblemHanoi`.

In [13]:
from aima_libs.hanoi_states import ProblemHanoi

En el problema definimos el estado inicial desde donde arrancamos:

![estados_hanoi_initial](./img/state_hanoi2.png)

In [14]:
varilla_izquierda = [5, 4, 3, 2, 1]
varilla_medio = []
varilla_derecha = []

initial_state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Al estado objetivo que queremos llegar:

![estados_hanoi_goal](./img/state_hanoi3.png)

In [15]:
varilla_izquierda = []
varilla_medio = []
varilla_derecha = [5, 4, 3, 2, 1]
goal_state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

Con estos estados, definamos el problema:

In [16]:
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

Con esta clase tenemos dos atributos:
* `initial`: Es el estado inicial
* `goal`: Es el estado objetivo.

Y cuatro métodos:
* `actions`:  Devuelve todas las acciones posibles que se pueden ejecutar desde un estado dado.
* `result`: Calcula el nuevo estado después de aplicar una acción.
* `path_cost`: Calcula el costo del camino de ir de un estado a otro.
* `goal_test`: Verifica si un estado particular es el estado objetivo.

Entonces, así podemos ver todas las acciones que podemos aplicar desde un estado dado:

In [17]:
varilla_izquierda = []
varilla_medio = [5, 3, 1]
varilla_derecha = [4, 2]
state = StatesHanoi(varilla_izquierda, varilla_medio, varilla_derecha, max_disks=5)

In [18]:
lista_acciones = problem.actions(state)
for action in lista_acciones:
    print(action)

Move disk 1 from 2 to 1
Move disk 1 from 2 to 3
Move disk 2 from 3 to 1


Aplicamos una de las acciones que nos devuelve:

In [19]:
# Aplicamos la acción de Mover el disco 1 de 2 a 3
new_state = problem.result(state=state, action=lista_acciones[1])

print(new_state)

HanoiState:  | 5 3 | 4 2 1


Es decir, fuimos de este estado:

![estados_hanoi_goal](./img/state_hanoi0.png)

a este estado:

![estados_hanoi_goal](./img/state_hanoi4.png)

pero ahora en el contexto del **Problema**.

Acumulemos el costo que venimos llevando al nuevo estado:

In [20]:
print(f"El costo acumulado según el problema del nuevo estado es {problem.path_cost(c=1, state1=state, action=lista_acciones[1], state2=new_state)}")

El costo acumulado según el problema del nuevo estado es 1.0


Apliquemos al nuevo estado otra acción:

In [21]:
lista_acciones = problem.actions(new_state)
for action in lista_acciones:
    print(action)

Move disk 3 from 2 to 1
Move disk 1 from 3 to 1
Move disk 1 from 3 to 2


In [22]:
# Aplicamos la acción de Mover el disco 3 de 2 a 1
new_state_2 = problem.result(state=new_state, action=lista_acciones[0])

print(new_state_2)

HanoiState: 3 | 5 | 4 2 1


Y ahora el costo acumulado es 2, dado que pasamos por dos estados para llegar a este nuevo estado:

In [23]:
print(f"El costo acumulado según el problema del nuevo estado es {problem.path_cost(c=1, state1=new_state, action=lista_acciones[0], state2=new_state_2)}")

El costo acumulado según el problema del nuevo estado es 2.0


Por último podemos ver si un estado dado es la solución:

In [24]:
if not problem.goal_test(state=new_state_2):
    print(f"{new_state_2} no es la solución final {goal_state}")

HanoiState: 3 | 5 | 4 2 1 no es la solución final HanoiState:  |  | 5 4 3 2 1


Con esta implementación ya tenemos la posibilidad de generar el grafo de estados de Hanoi,

![grafo_de_hanoi](./img/state_hanoi_graph.png)